주의 사항
1) selenium driver 업데이트 권장드립니다 -> !pip install selenium --upgrade  
2) 크롬창 우측 상단 점 3개 클릭 - 도움말 - chrome 정보에서 chrome 버전 확인하시고 거기에 맞는 웹드라이버 사용 부탁드립니다.  
3) 제가 직접 올린 웹드라이버는 1.16버전으로, 최신 1.15 크롬에서 원할하게 돌아감을 확인했습니다.  

pip install selenium  
pip install request  
pip install webdriver_manager  
등등 필요한 라이브러리 설치 후 실행 요망

In [48]:
import pandas as pd
import numpy as np
import random
import time

from bs4 import BeautifulSoup as bs
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from concurrent.futures import ThreadPoolExecutor, as_completed

## 웹드라이버 옵션 추가: 동적 크롤링 성능 향상을 위해 쓸데없는 옵션들 모두 제외
service = Service()
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

# headless 해제시에만 적용됨. 테스트 위해 추가.
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 
                                                    'popups': 2, 'geolocation': 2, 'notifications' : 2, 
                                                    'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 
                                                    'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 
                                                    'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 
                                                    'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 
                                                    'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 
                                                    'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 
                                                    'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)


## 옵션 1: 싱글 스레드 버전.
def crawl(url):

    ## 코멘트 저장 위해 빈 리스트 생성
    comment_list = []
    
    ## 정적 크롤링으로 화제글 위에서부터 10개 뽑아오기
    html_table = requests.get(url).text
    soup_table = bs(html_table,'html.parser')
    data_box = soup_table.find_all('a',attrs = {'class':'subject-link'})[:10]
    
    ## 드라이버 생성
    driver = webdriver.Chrome(service=service, options=options)

    for index, data in enumerate(data_box):
        
        print(index+1,"번째 게시글에서 댓글을 추출중입니다...")
        print("-------------------------------------------------------")
        
        ## 데이터박스 아이템 별로 링크 뽑아서 열기
        link = data.attrs['href']
        driver.get(link)

        try:
            ## 코멘트 래퍼 로딩까지 기다리기
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((
                By.XPATH,'//*[@id="powerbbsCmt2"]/div[2]')))
                
        except TimeoutException:
            print("요소가 나타나지 않았습니다.")
            
        else:
            ## Xpath 형식 동일한 댓글들 모조리 긁어오기
            comment_box = driver.find_elements(
                By.XPATH,'//*[starts-with(@id, "cmt")]/div[2]/div[2]/span')
                
            ## 전체 댓글에서 3개 랜덤추출 후 결과 리스트에 추가
            selected_comments = random.sample(comment_box, 3)
            comment_list = comment_list + [comment.text for comment in selected_comments]
    
    ## 드라이버 닫기
    driver.close()
                        
    print("댓글 추출이 완료되었습니다!")
    print("-------------------------------------------------------")
    return comment_list

"""
## 옵션 2: 멀티스레드 버전.
def crawl_single(link):
    print("게시글에서 댓글을 추출중입니다...")
    print("-------------------------------------------------------")
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(link)

    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((
            By.XPATH,'//*[@id="powerbbsCmt2"]/div[2]')))
    except TimeoutException:
        print("요소가 나타나지 않았습니다.")
    else:
        comment_box = driver.find_elements(
            By.XPATH,'//*[starts-with(@id, "cmt")]/div[2]/div[2]/span')
        selected_comments = random.sample(comment_box, 3)
        comment_list = [comment.text for comment in selected_comments]
        
    driver.close()

    print("댓글 추출이 완료되었습니다!")
    print("-------------------------------------------------------")
    return comment_list


def crawl(url):
    comment_list = []
    html_table = requests.get(url).text
    soup_table = bs(html_table, 'html.parser')
    data_box = soup_table.find_all('a', attrs={'class': 'subject-link'})[10:20]
    
    # ThreadPoolExecutor를 사용하여 병렬 처리
    with ThreadPoolExecutor(max_workers=10) as executor:
        links = [data.attrs['href'] for data in data_box]
        results = executor.map(crawl_single, links)
        for result in results:
            comment_list.extend(result)      
    return comment_list
"""

def query():

    while(True):
        print("인벤 코드를 이용해서 검색해주세요")
        print("-------------------------------------------------------")
        print("리그 오브 레전드 :              0")
        print("메이플스토리     :              1")
        print("피파온라인 4     :              2")
        print("디아블로 4       :              3")
        print("로스트아크       :              4")
        print("-------------------------------------------------------")
    
        try:
            game = int(input("코드 입력 "))
            if game < 0 or game > 4:
                raise ValueError()
    
        except ValueError:
            print("잘못된 입력입니다. 코드를 확인해주세요.")
            print("-------------------------------------------------------")
        
        except Exception as e:
            print("알 수 없는 예외가 발생했습니다:", e)
            print("-------------------------------------------------------")
        else:
            break
    
    ##start = time.time()
    
    print("-------------------------------------------------------")
    if game == 0:
        print("리그 오브 레전드 인벤의 오늘의 화제글을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/lol/4625?my=chuchu"
        result = crawl(url)
        
    elif game == 1:
        print("메이플스토리 인벤의 30추글 목록을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/maple/5974?my=chuchu"
        result = crawl(url)
        
    elif game == 2:
        print("피파온라인 4 인벤의 오늘의 화제글을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/fifaonline4/3146?my=chu"
        result = crawl(url)
        
    elif game == 3:
        print("디아블로 4 인벤의 30추글 목록을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/diablo4/6025?my=chuchu"
        result = crawl(url)
        
    elif game == 4:
        print("로스트아크 인벤의 30추글 목록을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/lostark/4811?my=chuchu"
        result = crawl(url)
        
    df_result = pd.DataFrame(result, columns=['comments'])
    df_result['comments'] = df_result['comments'].str.replace('\n',' ')
    df_result[df_result['comments']==' ']="이미지만 있는 댓글입니다."
    df_result.to_csv('C:/Users/wjdgh/Desktop/comments.csv',encoding='utf-8-sig')
    print("csv파일 생성 완료.")
    ##end = time.time()
    ##print(f"{end - start:.5f} sec")


In [57]:
df = query()

인벤 코드를 이용해서 검색해주세요
-------------------------------------------------------
리그 오브 레전드 :              0
메이플스토리     :              1
피파온라인 4     :              2
디아블로 4       :              3
로스트아크       :              4
-------------------------------------------------------
코드 입력 3
-------------------------------------------------------
디아블로 4 인벤의 30추글 목록을 불러옵니다.
-------------------------------------------------------
1 번째 게시글에서 댓글을 추출중입니다...
-------------------------------------------------------
2 번째 게시글에서 댓글을 추출중입니다...
-------------------------------------------------------
3 번째 게시글에서 댓글을 추출중입니다...
-------------------------------------------------------
4 번째 게시글에서 댓글을 추출중입니다...
-------------------------------------------------------
5 번째 게시글에서 댓글을 추출중입니다...
-------------------------------------------------------
6 번째 게시글에서 댓글을 추출중입니다...
-------------------------------------------------------
7 번째 게시글에서 댓글을 추출중입니다...
-------------------------------------------------------
8 번째 게시글에서 댓

In [58]:
df

,comments
0,rpg게임중 역대급 가격으로 발매하길래..기대치 겁나 상승했는데.. 4~5만짜리 패...
1,개똥철학이 강할거면 게임에 진정성이라도 있어야지..
2,엑조프라이멀 7만원 주고 샀었는데..
3,게임 만드는거 다 남자라고 이바보야 게임프로그래밍하고 게임만드는거 말이야 디아4가 ...
4,디아4 는 하면 할수록 진짜 게임이란걸 안해본 개발자들이 만들었구나?? 생각이 들긴...
5,환불 어려우니 초반 먹튀로 가는 거지
6,쟤들 본 뒤론 '게임 안해본 새키들이 디4를 논하냐' or '제대로 키워보지도 않고...
7,ㄹㅇㅋㅋ
8,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
9,맵에 장애물 같은게 혜자라는건가? 생각의 차이긴한데 혜자라는 표현은 좀..


In [40]:
df[df['comments']==' ']="이미지"